#IMPORTING THE LIBRARIES

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

#IMPORTING THE DATASET

In [ ]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:-1].values #columns 0,1,2 index does not provide any impact for the model. so we start from 3rd column and exclude the last column
y = dataset.iloc[:, -1].values

In [ ]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [ ]:
print(y)

[1 0 1 ... 1 1 0]


#Encoding the categorical data

In [ ]:
#Here there are 2 categorical data, gender and the country

#Label encoding the gender column

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2]) #it means all rows of the column index 2

In [ ]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


#One hot encoding the country column (because it contains more than 2 countries so we cannot use label encoder with just 0 and 1)

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


#Splitting dataset into train and test cell

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

#Feature scalling
***Whenever u apply neural network, u have to apply feature scalling.Have to scale every feature in the dataset and it is important for neural network***

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

#Part 2-Build the ANN

Initializing the ANN

In [ ]:
ann=tf.keras.models.Sequential()

Adding the input layer and the first hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

Adding the second hidden layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

Adding the ouput layer

In [ ]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
#sigmoid functions helps in providing predictions as well as probablities so it is recommended that the output layer uses a sigmoid function
#use activation softmax for output layer if there is more than 2 features to represent for classfication
#for regression, no activation function in the output layer

#Training the ANN

Compiling the ANN

In [ ]:
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
#optimizer updates the weight through stochastic gradient descent
#loss to compute difference between predictions and real
#metrics for accuracy
#binary_crossentropy for binary classfication and categorical_crossentropy for non binary classfication
#for non binary , the activation should be softmax in the output layer

Training the ANN on the training set

In [ ]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)
#batch size refers to number of predictions vs  the real results
#epochs refer to number of times the neural network is trained

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5537 - accuracy: 0.7445
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4581 - accuracy: 0.7961
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4390 - accuracy: 0.7976
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4265 - accuracy: 0.8043
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4146 - accuracy: 0.8123
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4031 - accuracy: 0.8145
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3921 - accuracy: 0.8175
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3827 - accuracy: 0.8195
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3751 - accuracy: 0.8216
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3682 - accura

#Making the predictions and evaluating the model

Predicting the result of a single observation

In [ ]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5) #0.5 is threshold for prediction

1/1 [==============================] - 0s 104ms/step
[[False]]


#Predicting the Test set results

In [ ]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 2ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


#Making the confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1526   69]
 [ 200  205]]


0.8655